In [1]:
%%capture
%pip install --upgrade iblm

In [2]:
%pip list | grep iblm

iblm                         0.3.30

[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
import sys
sys.path.append("../..")
import pandas as pd
import numpy as np

from src.iblm.ibl import IBLModel
from iblm import ibl

from src.data import pseudodata
from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_auc_score

### 円形データ

In [64]:
def circle_data(n_points, num_train_data, radius, noise):
    np.random.seed(0)
    theta = np.random.uniform(0, 2*np.pi, n_points)
    r = radius + np.random.normal(0, noise, n_points)  # ノイズを加える
    x0 = r * np.cos(theta)
    y0 = r * np.sin(theta)
    labels0 = np.zeros(n_points)

    # クラス1のデータ生成
    theta = np.random.uniform(0, 2*np.pi, n_points)
    r = radius * 0.5 + np.random.normal(0, noise, n_points)  # 内側の円で、ノイズを加える
    x1 = r * np.cos(theta)
    y1 = r * np.sin(theta)
    labels1 = np.ones(n_points)

    # データセットを結合
    x = np.vstack((np.column_stack((x0, y0)), np.column_stack((x1, y1))))
    y = np.hstack((labels0, labels1)).astype(int)

    # データの順番をシャッフル
    indices = np.random.permutation(len(y))
    x = x[indices]
    y = y[indices]

    x = np.round(x, decimals=3)
    x = pd.DataFrame(x)

    x_train = x[0:num_train_data]
    x_test = x[num_train_data:]

    y_train = y[:num_train_data]
    y_test = y[num_train_data:]

    return x_train, x_test, y_train, y_test


radius=10
noise=0.1

x_train, x_test, y_train, y_test = circle_data(n_points=sample, num_train_data=num_train_data, radius=radius, noise=noise)


### SVMデータセット

In [58]:
def svm_dataset(n_points, num_train_data):
    np.random.seed(0)  # 再現性のためのシード設定

    # クラス1とクラス2のデータ生成
    x1_1 = np.random.normal(loc=0.5, scale=0.5, size=n_points)
    x2_1 = np.random.normal(loc=0.5, scale=0.5, size=n_points)
    y1 = np.ones(n_points)  # クラス1のラベル

    x1_2 = np.random.normal(loc=-0.2, scale=0.5, size=n_points)
    x2_2 = np.random.normal(loc=-0.2, scale=0.5, size=n_points)
    y2 = np.zeros(n_points)  # クラス2のラベル

    # データの結合
    x1 = np.concatenate([x1_1, x1_2])
    x2 = np.concatenate([x2_1, x2_2])
    x = np.vstack([x1, x2]).T
    y = np.concatenate([y1, y2])

    # データのシャッフル
    indices = np.random.permutation(n_points * 2)
    x_shuffled = x[indices]
    y_shuffled = y[indices]

    # pandas DataFrameへの変換
    x_shuffled = pd.DataFrame(x_shuffled)

    # トレーニングセットとテストセットに分割
    x_train = x_shuffled.iloc[:num_train_data]
    x_test = x_shuffled.iloc[num_train_data:]
    y_train = y_shuffled[:num_train_data]
    y_test = y_shuffled[num_train_data:]

    return x_train, x_test, y_train, y_test


x_train, x_test, y_train, y_test = svm_dataset(n_points = sample, num_train_data = num_train_data)


### GBDTデータセット

In [87]:
def lgbm_dataset(n_points, num_train_data):
    np.random.seed(42)

    # Generate a mix of sine and cosine functions with noise as features
    X1 = np.linspace(0, 6*np.pi, n_points)
    X2 = np.sin(X1) + np.random.normal(0, 0.1, n_points)
    X3 = np.cos(X1) + np.random.normal(0, 0.1, n_points)
    X4 = X2 * X3 + np.random.normal(0, 0.1, n_points)  # Interaction term
    x = np.vstack((X2, X3, X4)).T

    # Generate labels based on a non-linear combination of features
    y = (np.sin(X1) > 0).astype(int)

    # Fix: Corrected the way of shuffling data using a single sequence of indices
    indices = np.arange(n_points)  # Fixed to generate indices for the actual number of points
    np.random.shuffle(indices)
    x_shuffled = x[indices]
    y_shuffled = y[indices]

    # Conversion to pandas DataFrame
    x_shuffled = pd.DataFrame(x_shuffled, columns=['Feature1', 'Feature2', 'Feature3'])

    # Splitting into training and test sets
    x_train = x_shuffled.iloc[:num_train_data]
    x_test = x_shuffled.iloc[num_train_data:]
    y_train = y_shuffled[:num_train_data]
    y_test = y_shuffled[num_train_data:]
    return x_train, x_test, y_train, y_test

x_train, x_test, y_train, y_test = lgbm_dataset(n_points = sample, num_train_data = num_train_data)

### sklearn擬似データ

In [8]:
num_train_data = 500 # 50〜300
num_test_data = 1000
sample = num_train_data + num_test_data
n_informative = 2
n_redundant = 0
n_features = n_informative + n_redundant
weights = [0.5, 0.5]
flip_y=0
seed = 3655  # 3655,3656,3657

# testデータの個数を揃えるためにtrain_test_splitを使っていない
x, y = make_classification(
    n_samples = sample,  # データ数
    n_features = n_features,  # 特徴量の数
    n_informative = n_informative,  # ラベル予測に意味のある特徴量の数
    n_redundant = n_redundant,  # 冗長な特徴量
    weights = weights,  # [0,1]の割合
    flip_y = flip_y, # 逆のラベルに反転する割合
    random_state = seed
    )

x = np.round(x, decimals=3)
x = pd.DataFrame(x)

x_test = x[0:num_test_data]
x_train = x[num_test_data:]

y_test = y[:num_test_data]
y_train = y[num_test_data:]

### GPT-4 Turbo

In [9]:
iblm = ibl.IBLModel(api_type="openai", model_name="gpt-4-0125-preview", objective="binary")
model = iblm.fit(x_train, y_train, seed=3655)
y_pred = iblm.predict(x_test)
iblm.evaluate(y_test, y_pred)

2024-02-15 21:21:20,455 [iblm.ibl][INFO] (ibl:ibl.py:fit:151)


{'roc_auc': 0.9918074756784434,
 'pr_auc': 0.9893431639830674,
 'accuracy': 0.81,
 'recall': 1.0,
 'precision': 0.7262247838616714,
 'f1_score': 0.8414023372287145}

In [10]:
print(model)

#########
def predict(x):
    import numpy as np

    df = x.copy()
    output = []
    for index, row in df.iterrows():
        # Do not change the code before this point.
        # Please describe the process required to make the prediction below.
        
        # Simple logistic regression coefficients derived from the dataset manually or through a simple algorithm
        # These coefficients are placeholders and should ideally be determined through analysis of the dataset
        # For demonstration purposes only
        intercept = 0.5
        coef_col0 = 0.8
        coef_col1 = -0.3
        
        # Logistic regression formula
        z = intercept + (coef_col0 * row[0]) + (coef_col1 * row[1])
        y = 1 / (1 + np.exp(-z))

        # Do not change the code after this point.
        output.append(y)
    return np.array(output)
#########


In [7]:
print(model)

#########
def predict(x):
    import numpy as np

    df = x.copy()
    output = []
    for index, row in df.iterrows():
        # Simple logistic regression coefficients are manually estimated based on the dataset characteristics.
        # These coefficients are placeholders and should ideally be determined using a logistic regression model fitting process.
        # Intercept
        b0 = -0.5
        # Coefficients for the features
        b1 = 0.8
        b2 = 0.2

        # Logistic regression model
        z = b0 + b1*row[0] + b2*row[1]
        y = 1 / (1 + np.exp(-z))

        output.append(y)
    return np.array(output)
#########


### GPT-3.5

In [199]:
iblm = ibl.IBLModel(api_type="openai", model_name="gpt-3.5-turbo-0125", objective="binary")
model = iblm.fit(x_train, y_train, seed=3655)
y_pred = iblm.predict(x_test)
iblm.evaluate(y_test, y_pred)

2024-02-12 06:07:48,926 [iblm.ibl][INFO] (ibl:ibl.py:fit:154)


{'roc_auc': 0.24206596888803533,
 'pr_auc': 0.36019643762895803,
 'accuracy': 0.317,
 'recall': 0.3333333333333333,
 'precision': 0.3159922928709056,
 'f1_score': 0.32443125618199803}

### Gemini Pro

In [11]:
iblm = ibl.IBLModel(api_type="gemini", model_name="gemini-pro", objective="binary")
model = iblm.fit(x_train, y_train)
y_pred = iblm.predict(x_test)
iblm.evaluate(y_test, y_pred)

2024-02-15 21:26:00,737 [iblm.ibl][INFO] (ibl:ibl.py:fit:151)


{'roc_auc': 0.8604810707885304,
 'pr_auc': 0.8908559550390702,
 'accuracy': 0.755,
 'recall': 0.751984126984127,
 'precision': 0.7595190380761523,
 'f1_score': 0.7557328015952144}

In [12]:
print(model)

#########
def predict(x):
    import numpy as np

    df = x.copy()
    output = []
    for index, row in df.iterrows():
        # Do not change the code before this point.
        # Please describe the process required to make the prediction below.
        y = 1 / (1 + np.exp(-(row[0] + row[1])))


        # Do not change the code after this point.
        output.append(y)
    return np.array(output)
#########


### ロジスティック回帰

In [201]:
log_reg = LogisticRegression(random_state=3655)
log_reg.fit(x_train, y_train)
log_reg_probs = log_reg.predict_proba(x_test)[:, 1]
log_reg_auc = roc_auc_score(y_test, log_reg_probs)


log_reg_preds = log_reg.predict(x_test) # 予測ラベルを計算
log_reg_accuracy = accuracy_score(y_test, log_reg_preds) # 精度を計算

# AUCと精度を出力
print(f"Logistic Regression AUC: {log_reg_auc:.4f}")
print(f"Logistic Regression Accuracy: {log_reg_accuracy:.4f}")

Logistic Regression AUC: 0.9724
Logistic Regression Accuracy: 0.9360


### SVM

In [202]:
svm = SVC(probability=True, random_state=42)  # probability=Trueで確率出力を有効化
svm.fit(x_train, y_train)
svm_probs = svm.predict_proba(x_test)[:, 1]
svm_auc = roc_auc_score(y_test, svm_probs)

# 追加コード: 精度を計算
svm_preds = svm.predict(x_test) # 予測ラベルを計算
svm_accuracy = accuracy_score(y_test, svm_preds) # 精度を計算

# AUCと精度を出力
print(f"SVM AUC: {svm_auc:.4f}")
print(f"SVM Accuracy: {svm_accuracy:.4f}")

SVM AUC: 0.9807
SVM Accuracy: 0.9360


### GBDT

In [203]:
gb_clf = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
gb_clf.fit(x_train, y_train)
y_pred = gb_clf.predict_proba(x_test)[:, 1]
gbdt_auc = roc_auc_score(y_test, y_pred)

# 追加コード: 精度を計算
gb_clf_preds = gb_clf.predict(x_test) # 予測ラベルを計算
gb_clf_accuracy = accuracy_score(y_test, gb_clf_preds) # 精度を計算

# AUCと精度を出力
print(f"GBDT AUC: {gbdt_auc:.4f}")
print(f"GBDT Accuracy: {gb_clf_accuracy:.4f}")

GBDT AUC: 0.9814
GBDT Accuracy: 0.9240


In [234]:
#values = [0.99180, 0.93893, 0.92402, 0.93959]
#values = [0.87694, 0.881488, 0.95156, 0.90104]
#values = [0.860481, 0.841916, 0.862391, 0.8450683]


#values = [0.72598, 0.85823, 0.85836, 0.91805, 0.99180]
#values = [0.75880, 0.90408, 0.88149, 0.93893, 0.83601]
#values = [0.67326, 0.92402, 0.650133, 0.223092, 0.85594]
#values = [0.93959, 0.90104, 0.72344, 0.845402, 0.91754]

#values = [0.75947, 0.87694, 0.809129, 0.29116, 0.706267]
#values = [0.12979, 0.80999, 0.881488, 0.84443, 0.32652]
#values = [0.29293, 0.931014, 0.95156, 0.85095, 0.727106]
#values = [0.68948, 0.90104, 0.310713, 0.84540, 0.2420659]

#values = [0.8128, 0.85822, 0.809121, 0.859184, 0.860481]
#values = [0.84044, 0.80999, 0.796973, 0.841916, 0.836025]
#values = [0.862391, 0.83288, 0.834107, 0.85095, 0.855949]
values = [0.844427, 0.818141, 0.84441, 0.84540, 0.8450683]

In [235]:
mean_value = sum(values) / len(values)
print(mean_value)
# 各値の平均値からの差の二乗の平均（分散）を計算
variance = sum((x - mean_value) ** 2 for x in values) / len(values)

# 分散の平方根を取って標準偏差を求める
standard_deviation = variance ** 0.5
print(standard_deviation)


0.83948926
0.010680877287489081
